# Policy Gradient methods

We will start with the standard policy gradient algorithm. This is a batch algorithm, which means that we will collect a large number of samples per iteration, and perform a single update to the policy using these samples. Recall that the formula for policy gradient is given by

$$\nabla_{\theta}\mathbb{E}_{\pi_{\theta}}\Big[ \sum_{t=0}^T\gamma^t r_t \Big] = 
\mathbb{E}_{\pi_{\theta}}\Big[ \sum_{t=0}^T \nabla_{\theta} \log\pi_{\theta}(a_t|s_t)\big(R_t - b(s_t)\big) \Big] \qquad\qquad (1)$$

- $\pi_{\theta}$ is a stochastic policy parameterized by $\theta$;
- $\gamma$ is the discount factor;
- $s_t$, $a_t$ and $r_t$ are the state, action, and reward at time $t$;
- $T$ is the length of a single episode;
- $b(s_t)$ is any funcion which does not depend on the current action $a_t$, and is called baseline;
- $R_t$ is the discounted cumulative future return (already defined in the DQN exercise);
Instead of optimizing this formula, we will optimize a sample-based estimation of the expectation, based on $N$ trajectories. For this you will first implement a function that computes $\log\pi_{\theta}(a_t|s_t)$ given any $s,~a$. 

In [1]:
#!/usr/bin/env python
import numpy as np
import gym
from simplepg.simple_utils import gradient_check, log_softmax, softmax, weighted_sample, include_bias, test_once, nprs
import tests.simplepg_tests
import math

## Constructing a stochastic policy

Let's assume that $\pi_{\theta}$ is a Gaussian with unit variance $\Sigma=I$ and mean $\mu=NN_{\theta}(s)$, where $NN_{\theta}$ is a Neural Network parameterized by $\theta$.

### 1. Create a Linear NN
Use two hidden linear layer with 256 hidden units and ReLu non-linearity, and use a linear output layer with no output non-linearity.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd

class MLP(nn.Module):
    def __init__(self, obs_size, act_size):
        super(MLP, self).__init__()
        #"*** YOUR CODE HERE ***"
        self.linear1 = nn.Linear(obs_size, 256)
        self.linear2 = nn.Linear(256,256)
        self.linear3 = nn.Linear(256, act_size)
    def forward(self, obs):
        #"*** YOUR CODE HERE ***"
        out = F.relu(self.linear1(obs))
        out = F.relu(self.linear2(out))
        out = self.linear3(out)
        return out

### 2. Create a Gaussian MLP policy
For Policy Gradient methods the policy needs to be stochastic. In our case, we will assume the distribution is a Gaussian where the mean $\mu_{\theta}(o)$ is the output of an MLP given the observation $o$ and unit variance. You will need to implement the `get_action` method that, given an observation $o$, samples an action $a$ from $\mathcal{N}(\mu_{\theta}(o),I)$; and the `get_logp_action` that gives the logprobability of a given action $a$ under the policy when observation $o$ is inputed. Remember the probability of a $n$-dimensional multivariate Gaussian with unit variance can be written as:
$$\frac{1}{\sqrt{(2\pi)^{n}}}\exp^{-\frac{1}{2}(a-\mu_{\theta}(o))^T(a-\mu_{\theta}(o))}$$

In [3]:
class GaussianMLP_Policy(object):
    def __init__(self, obs_size, act_size, NN):
        self.NN = NN(obs_size, act_size)
        
    def get_action(self, obs, rng=np.random):
        #"*** YOUR CODE HERE ***"
        obs_var = autograd.Variable(torch.from_numpy(obs).float(), requires_grad=False)
        mean = self.NN(obs_var).data.numpy()
        cov = np.identity(mean.shape[0])
        sampled_action = rng.multivariate_normal(mean, cov)
        return sampled_action
    
    def get_logp_action(self, obs, action):
        #"*** YOUR CODE HERE ***"
        # obs: Variable
        # action: Variable
        # log_p: Variable
        mean_var = self.NN(obs)
        n = action.size(1)
        diff = action - mean_var
        power = -0.5 * torch.sum(torch.pow(diff ,2.0) ,1)
        log_p = torch.log(torch.exp(power) / math.sqrt(pow(2*math.pi, float(n))))
        return log_p

### 3. Compute time-based baselines
Any function that does not depend on the action can be used as a baseline. The most usual one is to have a state-based baseline. In our case we will keep a simple time-based baseline that is the average return obtained at that particular time-step accross all paths collected in the previous iteration.

In [4]:
def compute_baselines(all_returns):
    baselines = np.zeros(len(all_returns))
    for t in range(len(all_returns)):
        #"*** YOUR CODE HERE ***"
        # Update the baselines
        # all_returns: list of lists. all_returns[time_step][path]
        baselines[t] = np.mean(all_returns[t]) if len(all_returns[t])>0 else 0
    return baselines

### 4. Compute returns
Given the rewards obtained in a path, return the discounted returns with the formula:
$$
R_t = \begin{cases}
r_t +\gamma R_{t+1} \qquad\text{ if non-terminal transition}\\
r_t \qquad\qquad\quad \text{ for terminal transition}
\end{cases} \qquad\qquad (2)
$$

In [5]:
def compute_returns(discount, rewards):
    returns = np.zeros_like(rewards)
    #"*** YOUR CODE HERE ***"
    for i in np.arange(len(returns)-1,-1,-1):
        returns[i] = rewards[i] + (0 if i == len(returns)-1 else discount*returns[i+1])
    return returns

## Run the algorithm
You are only asked to implement the surrogate reward that we take the gradient of. We do so by approximating the expectation in Eq. (1) by a sum over paths. In other words, the surrogate function can be written as:
$$ \sum_{i=0}^N\sum_{t=0}^{T_i} \log\pi_{\theta}(a^i_t|s^i_t)\big(R^i_t - b(t)\big) \qquad\qquad (3)$$
If you implemented it correctly, the reward should reach arround -20 in about 50 iterations.

In [6]:
from simplepg import point_env
env = gym.make('Point-v0')
obs_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

# Store baselines for each time step.
timestep_limit = env.spec.timestep_limit
baselines = np.zeros(timestep_limit)

# instantiate the policy
policy = GaussianMLP_Policy(obs_dim, action_dim, MLP)

[2018-04-23 22:22:39,338] Making new env: Point-v0


In [7]:
n_itrs = 50
batch_size = 2000
discount = 0.99
learning_rate = 0.1
render = False  # True  # see setup_instructions.pdf to render point-mass policy
natural_step_size = 0.01

# Policy training loop
for itr in range(n_itrs):
    # Collect trajectory loop
    n_samples = 0
    policy.NN.zero_grad()
    episode_rewards = []

    # Store cumulative returns for each time step
    all_returns = [[] for _ in range(timestep_limit)]

    all_observations = []
    all_actions = []
    all_centered_cum_rews = []

    while n_samples < batch_size:
        observations = []
        actions = []
        rewards = []
        ob = env.reset()
        done = False
        # Only render the first trajectory
        render_episode = n_samples == 0
        # Collect a new trajectory
        while not done:
            action = policy.get_action(ob)
            next_ob, rew, done, _ = env.step(action)
            observations.append(ob)
            actions.append(action)
            rewards.append(rew)
            ob = next_ob
            n_samples += 1
            if render and render_episode:
                env.render()
                
        # Go back in time to compute returns 
        returns = compute_returns(discount, rewards)
        # center the rewards by substracting the baseline
        centered_cum_rews = returns - baselines[:len(returns)]
        # save them in all_returns to compute time-based baseline for next iteration 
        for t, r in enumerate(returns):
            all_returns[t].append(r)

        episode_rewards.append(np.sum(rewards))
        all_observations.extend(observations)
        all_actions.extend(actions)
        all_centered_cum_rews.extend(centered_cum_rews)
    
    # autodiff loss
    obs_vars = autograd.Variable(torch.Tensor(all_observations), requires_grad=False)
    act_vars = autograd.Variable(torch.Tensor(all_actions), requires_grad=False)
    centered_cum_rews_vars = autograd.Variable(torch.Tensor(all_centered_cum_rews), requires_grad=False)
    
    logps = policy.get_logp_action(obs_vars, act_vars)
        
    #"*** YOUR CODE HERE ***"
    surr_loss = torch.sum(logps * centered_cum_rews_vars)
    
    surr_loss.backward()
    
    flat_grad = np.concatenate([p.grad.data.numpy().reshape((-1)) for p in policy.NN.parameters()])
    grad_norm = np.linalg.norm(flat_grad)
    
    for p in policy.NN.parameters():
        # roughly normalize gradiend and take step
        p.data += learning_rate * p.grad.data / (grad_norm + 1e-8)

    test_once(compute_baselines)

    baselines = compute_baselines(all_returns)
    
    print("Iteration: %d AverageReturn: %.2f GradNorm: %.2f" % (
    itr, np.mean(episode_rewards), grad_norm))

Test for __main__.compute_baselines passed!
Iteration: 0 AverageReturn: -41.47 GradNorm: 4683.90
Iteration: 1 AverageReturn: -39.78 GradNorm: 1197.01
Iteration: 2 AverageReturn: -39.59 GradNorm: 887.91
Iteration: 3 AverageReturn: -36.59 GradNorm: 1195.10
Iteration: 4 AverageReturn: -36.08 GradNorm: 997.15
Iteration: 5 AverageReturn: -34.29 GradNorm: 1128.68
Iteration: 6 AverageReturn: -32.95 GradNorm: 1409.37
Iteration: 7 AverageReturn: -32.17 GradNorm: 1654.11
Iteration: 8 AverageReturn: -31.85 GradNorm: 2034.69
Iteration: 9 AverageReturn: -28.44 GradNorm: 1100.82
Iteration: 10 AverageReturn: -28.17 GradNorm: 1134.04
Iteration: 11 AverageReturn: -27.77 GradNorm: 1483.22
Iteration: 12 AverageReturn: -26.80 GradNorm: 839.32
Iteration: 13 AverageReturn: -25.60 GradNorm: 1296.70
Iteration: 14 AverageReturn: -23.46 GradNorm: 748.20
Iteration: 15 AverageReturn: -24.82 GradNorm: 1173.66
Iteration: 16 AverageReturn: -23.46 GradNorm: 786.92
Iteration: 17 AverageReturn: -22.39 GradNorm: 878.82
